In [1]:
import nltk
import time
from nltk.corpus import stopwords
import nltk.data
import torch
import pdfplumber
from sentence_transformers import SentenceTransformer, util

In [2]:
!pip install nltk

In [3]:
# Download NLTK data (if not already downloaded)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
# Load BERT models
model1 = SentenceTransformer('paraphrase-MiniLM-L3-v2')
model2 = SentenceTransformer('all-distilroberta-v1')
model3 = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [59]:
# def calculate_similarity(array_1, array_2):
#     # Combine array elements into strings
#     arr_1_str = ' '.join(map(str, array_1))
#     arr_2_str = ' '.join(map(str, array_2))

#     # Lemmatize data
#     w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#     lemmatizer = nltk.stem.WordNetLemmatizer()

#     def lemmatize_text(text):
#         return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

#     arr_1_lemm_str = lemmatize_text(arr_1_str)
#     arr_2_lemm_str = lemmatize_text(arr_2_str)

#     # Convert lemmatized data to lower case
#     arr_1_lemm_str = arr_1_lemm_str.lower()
#     arr_2_lemm_str = arr_2_lemm_str.lower()

#     # Text similarity scores obtained using 'paraphrase-MiniLM-L3-v2' BERT model
#     embd1 = model1.encode(arr_1_lemm_str, convert_to_tensor=True)
#     embd2 = model1.encode(arr_2_lemm_str, convert_to_tensor=True)
#     cosine_scores1 = util.pytorch_cos_sim(embd1, embd2)

#     # Text similarity scores obtained using BERT model 'all-distilroberta-v1'
#     embd1 = model2.encode(arr_1_lemm_str, convert_to_tensor=True)
#     embd2 = model2.encode(arr_2_lemm_str, convert_to_tensor=True)
#     cosine_scores2 = util.pytorch_cos_sim(embd1, embd2)

#     # Text similarity scores obtained using BERT model 'multi-qa-distilbert-cos-v1'
#     embd1 = model3.encode(arr_1_lemm_str, convert_to_tensor=True)
#     embd2 = model3.encode(arr_2_lemm_str, convert_to_tensor=True)
#     cosine_scores3 = util.pytorch_cos_sim(embd1, embd2)

#     # Calculate average similarity score
#     avg_similarity_score = (cosine_scores1.item() + cosine_scores2.item() + cosine_scores3.item()) / 3

#     return avg_similarity_score


In [5]:
import csv
import os

def calculate_similarity(csv_file, array_1, array_2):
    # Combine array elements into strings
    arr_1_str = ' '.join(map(str, array_1))
    arr_2_str = ' '.join(map(str, array_2))

    # Lemmatize data
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()

    def lemmatize_text(text):
        return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

    arr_1_lemm_str = lemmatize_text(arr_1_str)
    arr_2_lemm_str = lemmatize_text(arr_2_str)

    # Convert lemmatized data to lower case
    arr_1_lemm_str = arr_1_lemm_str.lower()
    arr_2_lemm_str = arr_2_lemm_str.lower()

    # Text similarity scores obtained using 'paraphrase-MiniLM-L3-v2' BERT model
    embd1 = model1.encode(arr_1_lemm_str, convert_to_tensor=True)
    embd2 = model1.encode(arr_2_lemm_str, convert_to_tensor=True)
    cosine_scores1 = util.pytorch_cos_sim(embd1, embd2)

    # Text similarity scores obtained using BERT model 'all-distilroberta-v1'
    embd1 = model2.encode(arr_1_lemm_str, convert_to_tensor=True)
    embd2 = model2.encode(arr_2_lemm_str, convert_to_tensor=True)
    cosine_scores2 = util.pytorch_cos_sim(embd1, embd2)

    # Text similarity scores obtained using BERT model 'multi-qa-distilbert-cos-v1'
    embd1 = model3.encode(arr_1_lemm_str, convert_to_tensor=True)
    embd2 = model3.encode(arr_2_lemm_str, convert_to_tensor=True)
    cosine_scores3 = util.pytorch_cos_sim(embd1, embd2)

    # Calculate average similarity score
    avg_similarity_score = (cosine_scores1.item() + cosine_scores2.item() + cosine_scores3.item()) / 3

    # Write to CSV file
    header = ['Sr. No', 'Question Paper 1', 'Question Paper 2', 'Similarity Score']
    data = {
        'Sr. No': 1,  # Assuming this is the first entry
        'Question Paper 1': arr_1_str,
        'Question Paper 2': arr_2_str,
        'Similarity Score': avg_similarity_score
    }

    # Check if the CSV file exists
    file_exists = os.path.exists(csv_file)

    with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)

        # Write header if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write data
        writer.writerow(data)

    return avg_similarity_score


In [6]:

def model(user_ques_1, user_ques_2):
    # Stop word removal
    stop = stopwords.words('english')
    ques_1_nstop = ' '.join([word for word in user_ques_1.split() if word not in stop])
    ques_2_nstop = ' '.join([word for word in user_ques_2.split() if word not in stop])

    # Tokenize the questions
    tok_ques_1 = nltk.word_tokenize(ques_1_nstop)
    tok_ques_2 = nltk.word_tokenize(ques_2_nstop)
    tok_ques_1_str = ' '.join(map(str, tok_ques_1))
    tok_ques_2_str = ' '.join(map(str, tok_ques_2))

    # Lemmatize data
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()

    def lemmatize_text(text):
        return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

    ques_1_lemm_str = lemmatize_text(tok_ques_1_str)
    ques_2_lemm_str = lemmatize_text(tok_ques_2_str)

    # Convert lemmatized data to lower case
    ques_1_lemm_str = ques_1_lemm_str.lower()
    ques_2_lemm_str = ques_2_lemm_str.lower()

    # Text similarity scores obtained using 'paraphrase-MiniLM-L3-v2' BERT model
    st = time.time()
    embd1 = model1.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model1.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores1 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Text similarity scores obtained using BERT model 'all-distilroberta-v1'
    st = time.time()
    embd1 = model2.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model2.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores2 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Text similarity scores obtained using BERT model 'multi-qa-distilbert-cos-v1'
    st = time.time()
    embd1 = model3.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model3.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores3 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Calculate average similarity score
    avg_similarity_score = (cosine_scores1.item() + cosine_scores2.item() + cosine_scores3.item()) / 3

    return avg_similarity_score


In [7]:
# Define a function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [31]:
C:\Users\shric\Downloads\same_q.pdf
C:\Users\shric\Downloads\same_q2.pdf
    
C:\Users\shric\Desktop\Seminar\partial_same_1.pdf
C:\Users\shric\Desktop\Seminar\partial_same_2.pdf
    
    
C:\Users\shric\Desktop\Seminar\test1.pdf
C:\Users\shric\Desktop\Seminar\test2.pdf

SyntaxError: unexpected character after line continuation character (1502714465.py, line 1)

In [8]:
# Prompt the user to enter the paths to the PDF files
pdf_path1 = input("Enter the path to the first PDF file: ")
pdf_path2 = input("Enter the path to the second PDF file: ")


# Extract text from both PDF files
question_paper1_text = extract_text_from_pdf(pdf_path1)
                                             
question_paper2_text = extract_text_from_pdf(pdf_path2)

Enter the path to the first PDF file: C:\Users\visha\OneDrive\Desktop\Seminar\pair\pair1.1.pdf
Enter the path to the second PDF file: C:\Users\visha\OneDrive\Desktop\Seminar\pair\pair2.1.pdf


In [9]:
print(question_paper1_text)
print("-----------------------------------------------------------")
print(question_paper2_text)

1. What is the capital of France?
2. Explain the process of photosynthesis.
3. Solve the quadratic equation: 2x^2 + 5x - 3 = 0.
4. Compare and contrast mitosis and meiosis.
5. Define the concept of entropy in thermodynamics.
-----------------------------------------------------------
1. Who is the author of the novel "Pride and Prejudice"?
2. Describe the steps involved in cellular respiration.
3. Solve the equation: log(x) = 2.
4. Compare and contrast DNA replication and transcription.
5. Define the concept of momentum in physics.


In [11]:
#question_paper1_text = "1. Define and explain Unified Modeling Language (UML) 2. Explain conceptual model of UML 3. Explain building blocks in UML. 4. Explain things and relationships in UML. 5. List and explain diagrams in UML. 6. Explain common mechanism in UML"

#question_paper2_text = "1. Define and explain Unified Modeling Language (UML) 2. Explain conceptual model of UML 3. Explain building blocks in UML. 4. .List and explain UML2.0 diagrams. 5. Explain object diagram in UML."

import re

# Use regular expressions to split the text into individual questions
arr1 = re.split(r'\d+\.', question_paper1_text)
arr2 = re.split(r'\d+\.', question_paper2_text)

# Remove any leading or trailing whitespace from the questions
arr1 = [question.strip() for question in arr1 if question.strip()]
arr2 = [question.strip() for question in arr2 if question.strip()]

# Create an array to store the questions
arr1 = arr1
arr2 = arr2

print("arr1:", arr1)
print("--------------------------------------------------------------------------------------------")
print("arr2:", arr2)
# Assume 'output.csv' is the CSV file path
csv_file = 'output.csv'

# Call the calculate_similarity_and_save function with the CSV file path
print(calculate_similarity(csv_file, arr1, arr2))

new_paper = []
# flag = 0

for q1 in arr1:
    for q2 in arr2:
        similarity = model(q1, q2)
        if similarity > 0.7:
            #print(similarity)
            new_paper.append(q1)
            
            break  # Break to avoid adding duplicates

# Add questions from question_paper2 that didn't have matches
for q2 in arr2:
    if q2 not in new_paper:
        new_paper.append(q2)
        
for q1 in arr1:
    if q1 not in new_paper:
        new_paper.append(q1)
print("--------------------------------------------------------------------------------------------")
# Print the new_paper array
print("new_paper:", new_paper)

arr1: ['What is the capital of France?', 'Explain the process of photosynthesis.', 'Solve the quadratic equation: 2x^2 + 5x - 3 =', 'Compare and contrast mitosis and meiosis.', 'Define the concept of entropy in thermodynamics.']
--------------------------------------------------------------------------------------------
arr2: ['Who is the author of the novel "Pride and Prejudice"?', 'Describe the steps involved in cellular respiration.', 'Solve the equation: log(x) =', 'Compare and contrast DNA replication and transcription.', 'Define the concept of momentum in physics.']
0.3541257282098134
--------------------------------------------------------------------------------------------
new_paper: ['Who is the author of the novel "Pride and Prejudice"?', 'Describe the steps involved in cellular respiration.', 'Solve the equation: log(x) =', 'Compare and contrast DNA replication and transcription.', 'Define the concept of momentum in physics.', 'What is the capital of France?', 'Explain the 

In [12]:
!pip install fitz
!pip install PyMuPDF


  Obtaining dependency information for configparser from https://files.pythonhosted.org/packages/81/a3/0e5ed11da4b7770c15f6f319abf053f46b5a06c7d4273c48469b7899bd89/configparser-6.0.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
     ---------------------------------------- 96.9/96.9 kB ? eta 0:00:00
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     - -------------------------------------- 0.1/3.3 MB 6.4 MB/s eta 0:00:01
     - -------------------------------------- 0.1/3.3 MB 6.4 MB/s eta 0:00:01
     --- ------------------------------------ 0.3/3.3 MB 2.3 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.3 MB 2.4 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.3 MB 2.4 MB/s eta 0:00:02
     -------- ------------------------------- 0.7/3.3 MB 2.4 MB/s eta 0:00:02
     -------- ------------------------------- 0.7/3.3 MB 2.5 MB/s eta 0:00:02
     ---------- -------------

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [286 lines of output]
  Reading version file C:\Users\visha\AppData\Local\Temp\pip-install-3etuorih\traits_fd0d8fcf07ab4eaeabf375ac4b6c8b1a\traits\version.py
  Package version from version file: ('6.3.2', '6df2ff9bd1d21c74e688aff6f67a19fbbefdd53b')
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\traits
  copying traits\api.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\base_trait_handler.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\constants.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\ctrait.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\editor_factories.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\has_traits.py -> build\lib.win-amd64-cpython-311\traits
  copying traits\interface_

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/2b/14/b3d950d252e64997a28bc2452d1fa19161888c80ee49cacee9bb06a9ec05/PyMuPDF-1.23.6-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.6 from https://files.pythonhosted.org/packages/f4/f6/487aba0931328119d8674637dc52b72d2f3b75ab373abd223436bb4bf605/PyMuPDFb-1.23.6-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.5 MB 5.0 MB/s eta 0:00:01
   ------ --------------------------------- 0.6/3.5 MB 5.8 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.5 MB 6.7 MB/s eta 0:00:01
   ---------------- ----------------------- 1.4/3.5 MB 7.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.9/3.5 MB 8.6 MB/s eta 0:00:01
   ----------------------------- ---------- 2.6/3.5 MB 9.1 MB/s eta 0:00:01
   -------------------------------------- - 3.3/3.5

In [13]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas


# Create a PDF document with A4 page size
pdf_file = "output.pdf"
c = canvas.Canvas(pdf_file, pagesize=A4)

# Set font and other attributes
c.setFont("Helvetica", 12)
line_height = 14  # Adjust line height as needed
bottom_margin = 20  # Margin to leave at the bottom of the page

# Calculate the available height on the page
page_height = A4[1] - bottom_margin

# Initialize page count
page_number = 1

# Iterate through the list and add the content to the PDF with pagination
y_position = page_height

for index, content in enumerate(new_paper, start=1):
    if y_position - line_height < 0:
        # If the content would overflow, start a new page
        c.showPage()
        c.setFont("Helvetica", 12)  # Reset font for the new page
        y_position = page_height
        page_number += 1

    content_with_index = f"{index}. {content}"
    c.drawString(20, y_position, content_with_index)
    y_position -= line_height

# Save the PDF document
c.save()


In [14]:
# Create a text file to write the content
txt_file = "output.txt"

# Open the text file for writing
with open(txt_file, "w") as file:
    for index, content in enumerate(new_paper, start=1):
        content_with_index = f"{index}. {content}\n"
        file.write(content_with_index)


In [15]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER
from reportlab.platypus import Spacer

output_file = 'output.pdf'

# Define custom page size with margins
page_width = letter[0]
page_height = letter[1]
left_margin = 36  # Adjust the margin values as needed
right_margin = 36
top_margin = 36
bottom_margin = 36

# Create a PDF document with custom margins
doc = SimpleDocTemplate(output_file, pagesize=(page_width, page_height), leftMargin=left_margin, rightMargin=right_margin, topMargin=top_margin, bottomMargin=bottom_margin)

# Create a list to store the content of the PDF
story = []

# Define a style for the content
styles = getSampleStyleSheet()
style = styles['Normal']
style.alignment = TA_CENTER  # Center align the tex

# Open and read the input text file
with open(txt_file, 'r') as file:
    for line in file:
        paragraph = Paragraph(line.strip(), style)
        story.append(paragraph)
        story.append(Spacer(1, 5))  # Add some space between paragraphs

# Build the PDF document
doc.build(story)

print(f'PDF file "{output_file}" generated successfully.')


PDF file "output.pdf" generated successfully.
